 # **Two-Dimensional Parity Checking**

In [283]:
import numpy as np
from numpy import ndarray
import random

Input (from keyboard) will be a string containing binary characters (input will be validated, and the length of the string must be a multiple of 7).

In [284]:
def remove_first_zeros(input_str:str) -> str:
    if input_str.startswith('0'):
        return input_str.lstrip('0')
    else:
        return input_str

def binary_characters(input_str: str) -> bool:
    return all(char in '01' for char in input_str)

def validate_input(input_str: str) -> bool:
    return len(input_str) % 7 == 0 and binary_characters(input_str)

message_str: str = '10101110101110'
#message_str = input("Enter message:")
message_str = remove_first_zeros(message_str)
ok: bool = False

while not ok:
    if not validate_input(message_str):
        new_message = input("Invalid input\nEnter new message: ")
        message_str = new_message
    else:
        print("Valid input.")
        print(message_str)
        ok = True

Valid input.
10101110101110


The message will be converted into a matrix with 7 columns and a variable number of lines (depending on the length of the entered message).

In [285]:
def convert_input_to_matrix(message: str, columns:int) -> ndarray:
    if validate_input(message) is False:
        raise ValueError("Invalid message length.")
    else:
        mat = np.array(list(message),  dtype=int).reshape(len(message)//columns, columns)
        return mat

message:ndarray = convert_input_to_matrix(message_str, 7)
print(message)

[[1 0 1 0 1 1 1]
 [0 1 0 1 1 1 0]]


Parity bits will be calculated on each line and each column as follows:
- for an even number of 1 on a line/column, a 0 will be added on the last column/line;
- for an odd number of 1, a 1 will be added;
- the significant bit is the bit in the lower-right corner of the matrix.

The resulting matrix will be displayed.

In [286]:
def message_matrix(message:ndarray) -> ndarray:
    new_mat = np.zeros((message.shape[0] + 1, message.shape[1] + 1), dtype=int)
    new_mat[:-1, :-1] = message
    new_mat[-1, :-1] = new_mat[:-1, :-1].sum(axis=0) % 2
    new_mat[:, -1] = np.sum(new_mat[:, :-1], axis=1) % 2
    return new_mat

n_message:ndarray = message_matrix(message)
print(n_message)

[[1 0 1 0 1 1 1 1]
 [0 1 0 1 1 1 0 0]
 [1 1 1 1 0 0 1 1]]


Message corruption will be simulated by randomly generating a position of a binary character of the message. Depending on the generated position, the bit on that position will change.

In [287]:
def corrupt_message(message:ndarray) -> ndarray:
    rows, cols = message.shape
    row, col = random.randint(0, rows - 2), random.randint(0, cols - 2)
    print("Corrupted Position:", (row, col))
    
    if(message[row,col] == 1):
        message[row,col] = 0
    else:
        message[row,col] = 1
    
    return message

corrupted_message = corrupt_message(n_message)
print(corrupted_message)


Corrupted Position: (1, 4)
[[1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0]
 [1 1 1 1 0 0 1 1]]


Identify the corrupted position using the two-dimensional parity check, and display the corrupted position

In [288]:
def two_dimensional_parity_check(message: ndarray) -> tuple:
    rows, cols = message.shape
    row_index, col_index = None, None
    
    for row in range(rows - 1): # check for rows
        row_ones = sum(message[row, :cols - 1])  
        if row_ones % 2 != message[row, cols - 1]:  
            row_index = row
            break
    
    for col in range(cols - 1): # check for columns
        col_ones = sum(message[:rows - 1, col]) 
        if col_ones % 2 != message[rows - 1, col]: 
            col_index = col
            break
    
    return (row_index, col_index) if row_index is not None and col_index is not None else None

def message_display(message:ndarray) -> None:
    if(two_dimensional_parity_check(message)):
        print("Corrupted message detected at position:", two_dimensional_parity_check(message), "\n")
    else:
        print("No corrupted message detected.\n")

print(message_matrix(message))
message_display(message_matrix(message))
print(corrupted_message)
message_display(corrupted_message)


[[1 0 1 0 1 1 1 1]
 [0 1 0 1 1 1 0 0]
 [1 1 1 1 0 0 1 1]]
No corrupted message detected.

[[1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0]
 [1 1 1 1 0 0 1 1]]
Corrupted message detected at position: (1, 4) 



In [289]:
del message, message_str, n_message, corrupted_message

# **CRC (Cyclic Redundancy Check)**

1.	Se introduce de la tastatura un sir de caractere binare si un polinom generator (cu coeficienti 0 si 1).
2.	Se fac urmatoarele verificari: sirurile sa fie binare si lungimea mesajului sa fie mai mare decat numarul de coeficienti ai polinomului generator.

In [290]:
def get_binary_input(prompt):
    while True:
        user_input = input(prompt)
        user_input = remove_first_zeros(user_input)
        if binary_characters(user_input):
            return user_input
        else:
            print("Invalid input. Please enter binary characters only.")

message_str = get_binary_input("Enter message: ")
coefficients = get_binary_input("Enter Polynomial: ")

while len(coefficients) >= len(message_str):
    print("Polynomial length should be smaller than message length.")
    coefficients = get_binary_input("Enter Polynomial: ")

print("Valid input.")
print("Message:", message_str)
print("Polynomial:", coefficients)


Valid input.
Message: 10101110101110
Polynomial: 1011


3.	Se extinde mesajul cu un numar de 0-uri egal cu gradul polinomului introdus.

In [291]:
def get_degree_of_polynomial(input:str) -> int:
    return len(input) - 1

degree = get_degree_of_polynomial(coefficients)

def extend_message(message:str, degree:int) -> str:
    return message + '0' * degree

message = extend_message(message_str, degree)
print(message)

10101110101110000


4.	Se efectueaza succesiv operatii de XOR intre mesajul extins si coeficientii polinomului pana cand lungimea restului obtinut este strict mai mica decat lungimea sirului de coeficienti.
5.	Se vor afisa rezultatele intermediare ale operatiei de XOR.

In [292]:
def XOR(first, second):
    if first == "0" and second == "1":
        return "1"
    if first == "1" and second == "0":
        return "1"
    return "0"

#  10101110101110
    
def compute_xor(message, coefficients, degree):
    intermediate_results = [] 
    while len(message) >= len(coefficients):
        xor_result = ''
        for i in range(degree + 1):
            xor_result += XOR(message[i], coefficients[i])
        message = xor_result + message[degree + 1:]
        intermediate_results.append(message) 
        message = remove_first_zeros(message)
        intermediate_results.append(message)
        if len(xor_result) < len(coefficients):
            break
    return xor_result, message, intermediate_results
        
message_c = message
xor_result, remaining_message, intermediate_results = compute_xor(message_c, coefficients, degree)
for result in intermediate_results:
    print(result)
print("Final:", remaining_message)

00011110101110000
11110101110000
01000101110000
1000101110000
0011101110000
11101110000
01011110000
1011110000
0000110000
110000
011100
11100
01010
1010
0001
1
Final: 1


6.	Se executa iarasi operatia de sau exclusiv intre mesajul extins si restul final obtinut, dar pozitionarea restului se va face sub finalul mesajului extins. 
7.	Acest rezultat se va afisa.